# TensorRTへの変換

# TensorRTでモデルを最適化する

[02_interactive regression.ipynb`)](./02_interactive_regression.ipynb). で、データセットの作成と、転移学習をおこないます。`road_following_model.pth`という名前(任意の名前でもOK)で、学習済みモデルが保存されていると思います。ここでは、tenserRTに変換する処理を実行します。


In [1]:
import torch
import torchvision

CATEGORIES = ['apex']

device = torch.device('cuda')
model = torchvision.models.resnet18(pretrained=False)
model.fc = torch.nn.Linear(512, 2 * len(CATEGORIES))
model = model.cuda().eval().half()

学習済みモデルをロードします。下記では、`road_following_model.pth`を読み込んでいますが、違う名前で保存した場合は、学習済みモデルの名前を書き直してください。

In [2]:
model.load_state_dict(torch.load('model.pth'))

<All keys matched successfully>

TensorRTでの推論を高速化するために、 ``torch2trt`` を使ってモデルを変換・最適化します。詳細を知りたい人は、 [torch2trt](https://github.com/NVIDIA-AI-IOT/torch2trt) を参考にしてください。

> この最適化の処理には数分間かかります。

In [3]:
from torch2trt import torch2trt

data = torch.zeros((1, 3, 224, 224)).cuda().half()

model_trt = torch2trt(model, [data], fp16_mode=True)

下記のセルで、最適化されたモデルを保存します。

In [4]:
torch.save(model_trt.state_dict(), 'model_trt.pth')

# 最適化されたモデルを走行させる

`road_following.ipynb`または、`road_following_live.ipynb` で最適化したモデルを読み込んで走らせます。